# NAVER TRENDS CRAWLING

* 공식 API 제공 (ID/SECRET 발급)
* 각 영화마다 키워드 설정하여 트랜드 조회 가능
* 기간별, 성별, 연령별, 채널별 조회 가능
* 동시 조회 가능 검색어 5개

[Reference](https://brunch.co.kr/@sukhyun9673/13)

### Load Package

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import urllib.request
import json

### Define Class

In [ ]:
class NaverTrend:
    def __init__(self):
        """
        NaverTrend Class
        """
        self.min_movie = []
        self.max_movie = []

    def set_api_info(self, api_info):
        self.client_id = api_info["client_id"]
        self.client_secret = api_info["client_secret"]
        self.url = api_info["url"]
        return

    def load_data(self, movie_data):
        self.movie_data_full = movie_data
        self.movie_list_full = movie_data.iloc[:,0]
        return

    def _define_body(self, movie_list):
        """
        movie_list: list
        TREND 조회 요청에 필요한 Body 파트를 정의한다.
         - 상세 조건 parameter로 받을 수 있도록 수정 필요
         - movie_list, keyword_list 쌍으로 받을 수 있도록 수정 필요 (DataFrame, Nested List, Two Series, ...)
        """
        keywords_list = ["{\"groupName\": \"" + movie + "\", \"keywords\": [\"" + movie + "\"]}" for movie in movie_list]
        body ="{\"startDate\":\"2018-12-01\", \"endDate\":\"2020-02-28\", \"timeUnit\":\"date\", \"keywordGroups\":" + "[" + ", ".join(keywords_list) + "]}"
        return(body)

    def _get_response_body(self, body):
        """
        body: string generated by _define body()
        Naver API로부터 트렌드 조회 결과를 받아온다.
        """
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id", self.client_id)
        request.add_header("X-Naver-Client-Secret", self.client_secret)
        request.add_header("Content-Type", "application/json")
        response = urllib.request.urlopen(request, data = body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            response_str = response.read().decode("utf-8") # repsonse body string
            response_dict = json.loads(response_str) # response body dict
            return(response_dict)
        else:
            print("Error Code:" + rescode)
    
    def _response_to_df(self, response_dict):
        """
        response_dict: dictionary generated by _get_response_body()
        response_body_dict를 DataFrame으로 변환한다.
        """
        trend_list = []
        for i in range(len(response_dict["results"])): # Need to find for loop alternatives.
            ttl = response_dict["results"][i]["title"]
            data = response_dict["results"][i]["data"]
            
            time = np.array([datetime.strptime(i["period"], "%Y-%m-%d") for i in data])
            value = np.array([j["ratio"] for j in data])
            title = np.repeat(ttl, len(value), axis=0)
            
            trend = pd.DataFrame({"Keyword": title, "Time": time, "TrendIndex": value})
            trend_list.append(trend)
        response_df = pd.concat(trend_list).reset_index(drop=True)
        return(response_df)

    def _recursive_search(self, init_cand, find_max=True):
        """
        init_cand: initial min/max candidates
        전체 리스트 중 검색 횟수가 최대/최소인 영화를 찾을 때까지 5개씩 TREND를 조회한다.
        """
        def _search(cand):
            n_movie = len(cand)
            rng = range(0, n_movie, 5)

            cand_subset = []
            for i, idx in zip(rng, range(len(rng))):
                movie_subset = [*cand[i:i+5]]
                trend_subset = self.get_trend(movie_subset)
                
                cand_idx = trend_subset['TrendIndex'].idxmax() if find_max == True else trend_subset['TrendIndex'].idxmin()
                cand_in_five = trend_subset.iloc[cand_idx]["Keyword"]  
                cand_subset.append(cand_in_five)
                print(cand_in_five, " in ", movie_subset)
            return(cand_subset)
        
        print("\n============================== CANDIDATES ==============================")
        cand = _search(init_cand)
        if len(cand) != 1:
            return(self._recursive_search(init_cand[init_cand.isin([*cand])]))
        else:
            return(cand)

    def get_trend(self, target, along_with = False):
        """
        특정 영화의 TREND를 조회한다. along_with = True일 경우, min/max moive와 함께 조회하여 스케일을 조정한다.
        """
        target_input = target
        if along_with == True:
            target = set([*target, *self.min_movie, *self.max_movie]) 
            print("Scale Info: max_movie: {0} | min_movie: {1} | target: {2}".format(self.max_movie, self.min_movie, target))
        body = self._define_body(set([*target]))
        response_dict_subset = self._get_response_body(body)
        response_subset = self._response_to_df(response_dict_subset)
        target_trend = response_subset[response_subset.Keyword.isin([*target_input])].reset_index(drop=True)
        return(target_trend)

    def set_movie(self, max = [], min = []):
        self.max_movie = self._recursive_search(self.movie_list_full, find_max = True) if max == True else max        
        self.min_movie = self._recursive_search(self.movie_list_full, find_max = False) if min == True else min
        return

---

### Execution

In [ ]:
api_info = {
    "client_id": "fe5kOiBVrMopPhGmaeeh",
    "client_secret": "zauzLV_BPn",
    "url": "https://openapi.naver.com/v1/datalab/search"
}

In [ ]:
api_info = {
    "client_id": "6GPEYsepjIil0OCSMnU5",
    "client_secret": "QUd5qt5QU9",
    "url": "https://openapi.naver.com/v1/datalab/search"
}

In [ ]:
## OLD FILE
# mv2019 = pd.read_table("~/crawling/data/movie2019.dat", sep = "|", header=None, names = ["Title", "Year", "Month", "Day"])
# open_date = mv2019.Year.str.replace("년", "") + "-" + mv2019.Month.str.replace("월", "").str.zfill(2) + "-" + mv2019.Day.str.replace("일", "").str.zfill(2)
# mv2019["OpenDate"] = [datetime.strptime(i, "%Y-%m-%d") for i in open_date]
# mv2019 = mv2019.drop(['Year', 'Month', 'Day'], axis=1)

In [ ]:
mv2019 = pd.read_table("~/MovieTrends/data/movie_list_200602.csv", sep = ",", encoding = "EUC-KR", header=0, names = ["Title", "Subtitle", "TitleEn", "OpenDate", "Count"])
mv2019.OpenDate = pd.Series([pd.Timestamp(datetime.strptime(i, '%Y%m%d')) for i in map(str, mv2019.OpenDate)])
mv2019 = mv2019[["Title", "OpenDate"]]

In [ ]:
NT = NaverTrend()
NT.set_api_info(api_info)
NT.load_data(mv2019)

In [ ]:
NT.set_movie(max = True)

In [ ]:
target_subset_list = [[*NT.movie_list_full[idx:idx+4]] for idx in [*range(0, len(NT.movie_list_full), 4)]]
result_list = [NT.get_trend(target_subset, True) for target_subset in target_subset_list]

In [ ]:
result_full = pd.concat(result_list)
result_full

In [ ]:
result_merged = result_full.merge(NT.movie_data_full, how="left", left_on="Keyword", right_on="Title").drop("Title", 1)
result_merged["D"] = result_merged.Time - result_merged.OpenDate
result_merged

In [ ]:
result_merged.to_csv("~/crawling/data/result_final.csv")

---

### Simple Plotting

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
mpl.rc('font', family='NanumGothic')
mpl.rc('figure', figsize=(15,10))
mpl.rc('axes', unicode_minus=False)

# ax = sns.lineplot(x='Time', y='Trendindex', hue='Keyword', data=result)
ax = sns.lineplot(x='D', y='TrendIndex', hue='Keyword', data=result_merged, legend=False)

plt.title('Line Graph by seaborn', fontsize=20)
plt.ylabel('Trend', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.legend(fontsize=12, loc='best')
plt.show()

---

### Check Scaling Method: MaxAbsScaling

In [ ]:
result_minmax = NT.get_trend(["백두산", "말모이", "시동", "82년생 김지영", "감쪽같은 그녀"])
result_min = NT.get_trend(["감쪽같은 그녀", "말모이", "시동", "82년생 김지영"])
result_max = NT.get_trend(["백두산", "말모이", "시동", "82년생 김지영"])
result_1 = NT.get_trend(["백두산", "말모이"])
result_2 = NT.get_trend(["백두산", "시동"])
result_3 = NT.get_trend(["백두산", "82년생 김지영"])

In [ ]:
cknx1 = result_minmax[result_minmax.Keyword == "말모이"].merge(result_1, how="left", left_on=["Keyword", "Time"], right_on=["Keyword", "Time"])
cknx2 = result_minmax[result_minmax.Keyword == "시동"].merge(result_2, how="left", left_on=["Keyword", "Time"], right_on=["Keyword", "Time"])
cknx3 = result_minmax[result_minmax.Keyword == "82년생 김지영"].merge(result_3, how="left", left_on=["Keyword", "Time"], right_on=["Keyword", "Time"])
sum(cknx1["TrendIndex_x"] != cknx1["TrendIndex_y"])
sum(cknx2["TrendIndex_x"] != cknx2["TrendIndex_y"])
sum(cknx3["TrendIndex_x"] != cknx3["TrendIndex_y"])

In [ ]:
ckx1 = result_max[result_max.Keyword == "말모이"].merge(result_1, how="left", left_on=["Keyword", "Time"], right_on=["Keyword", "Time"])
ckx2 = result_max[result_max.Keyword == "시동"].merge(result_2, how="left", left_on=["Keyword", "Time"], right_on=["Keyword", "Time"])
ckx3 = result_max[result_max.Keyword == "82년생 김지영"].merge(result_3, how="left", left_on=["Keyword", "Time"], right_on=["Keyword", "Time"])
sum(ckx1["TrendIndex_x"] != ckx1["TrendIndex_y"])
sum(ckx2["TrendIndex_x"] != ckx2["TrendIndex_y"])
sum(ckx3["TrendIndex_x"] != ckx3["TrendIndex_y"])

In [ ]:
ckn1 = result_min[result_min.Keyword == "말모이"].merge(result_1, how="left", left_on=["Keyword", "Time"], right_on=["Keyword", "Time"])
ckn2 = result_min[result_min.Keyword == "시동"].merge(result_2, how="left", left_on=["Keyword", "Time"], right_on=["Keyword", "Time"])
ckn3 = result_min[result_min.Keyword == "82년생 김지영"].merge(result_3, how="left", left_on=["Keyword", "Time"], right_on=["Keyword", "Time"])
sum(ckn1["TrendIndex_x"] != ckn1["TrendIndex_y"])
sum(ckn2["TrendIndex_x"] != ckn2["TrendIndex_y"])
sum(ckn3["TrendIndex_x"] != ckn3["TrendIndex_y"])

### The number of API Calls Needed to Find max_movie

#### Exact Number

In [ ]:
import math
def n_call(n_item, cnt = 0):
    temp = math.ceil(n_item/5)
    cnt = cnt + temp
    if temp == 1:
        return(cnt)
    else:
        return(n_call(temp, cnt))

In [ ]:
n_call(len(NT.movie_list_full))

#### Diff between the exact number of calls needed and approximation

In [ ]:
import random
d = [random.randint(100000000, 1000000000) for _ in range(10000)]
x = [n_call(i) for i in d]
y = [i/4 for i in d]

diff = [abs(i-j) for i, j in zip(x, y)]
sum(diff)/len(diff)

---

### 한글 폰트 문제 해결

In [ ]:
import matplotlib.font_manager as fm
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
font_list

In [ ]:
for font in fm.fontManager.ttflist:
    if 'Nanum' in font.name:
        print(font.name, font.fname)

In [ ]:
fm.FontProperties(fname=font_list[10]).get_name()

In [ ]:
mpl.get_cachedir()

In [ ]:
!rm '/home/jupyter-tshahn2020/.cache/matplotlib/fontlist-v310.json'

---